## Quantifying Linguistic Degeneration

Once I have the data, I must think about how to quantify linguistic degeneration. For this, I will be using the following metrics:
<br>

***Lexical Quality***

1. Type-token ratio (TTR): measures vocabulary diversity
2. Zipf curve tail heaviness: measures use of rare words
3. Average word length: measures use of larger words
4. Age of acquisition: measures word difficulty

***Syntactic Quality***

5. Parse tree depth: measures sentence complexity
6. Fragment ratio: measure of informality
7. Index of Syntactic Complexity: measures overall syntactic quality

***Orthographical Quality***

8. Levenshtein distance to dictionary: spelling errors
9. Punctuation frequency: attention to grammar

***Substantive Quality***

10. Abstract concepts
11. Figurative language

Note that I do not use other traditional measures of lexical complexity, such as ambiguity, vague quantifier frequency, orthographic neighborhood size, terminology inconsistency. Word ambiguity almost exclusively measures the complexity of the text from the perspective of the reader. As the purpose of this analysis is to look at language degeneration over time as a result of modern phenomena, I am principally interested in the intellect required to produce the complexity, meaning I am looking at true substantive complexity as opposed to inteprative complexity. Similarly, vague quantifier frequency, orthographic neighborhood size, and terminology inconsistency also measure complexity of interpratation, hence they are excluded. It is also worth noting that I categorize abstract concepts and figurative language under substantive quality as opposed to lexical complexity, as they involve more intimately the meaning of the word.

Likewise, I exclude certain measures of syntactic complexity, such as sentence length, information overload, passive voice, and negation. In the case of sentence length, it can be a sign of high (i.e., complexity) or low (i.e., run-on sentences) quality writing, which is why I have excluded it. Additionally, sentence complexity can be better measured via ICS. Information overload, passive voice, and negation are excluded for the same interpratability versus quality distinction in the above paragraph.

## Computing Lexical Quality Metrics With Python

In [46]:
import re
import numpy as np
import pandas as pd
from wordfreq import zipf_frequency

In [41]:
def tokenize(text):
    '''Helper function to tokenize a string of text, removing non-alphabetic characters'''
    text = text.lower()
    processed = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = processed.split()

    return tokens

example = "Nicholas $%#! @@is# $@! 123 gre4at.👍"
print(example)
print(tokenize(example))
print('The result should only contain words')

Nicholas $%#! @@is# $@! 123 gre4at.👍
['nicholas', 'is', 'great']
The result should only contain words


In [42]:
def ttr(text):
    tokens = tokenize(text)

    # error handling for when there are no tokens
    if len(tokens) == 0:
        return 0.0
    
    # recall that TTR is number of unique words / number of words
    num_types = len(set(tokens))
    num_tokens = len(tokens)
    ttr = num_types / num_tokens

    return ttr

example2 = "Nicholas is Nicholas"
print(ttr(example2))
print('The result should be 0.66')

0.6666666666666666
The result should be 0.66


In [43]:
def avg_word_length(text):
    '''Function that determines the average word length of a given text'''
    tokens = tokenize(text)

    # error handling for when there are no tokens
    if len(tokens) == 0:
        return 0.0
    
    average_length = np.mean([len(word) for word in tokens])

    return average_length

print(avg_word_length(example))
print('The result should be 5')

5.0
The result should be 5


In [ ]:
# build aoa_dict: word -> average age of acquisition
aoa_df = pd.read_csv("Data/KupermanAoAData.csv")
aoa_dict = dict(zip(aoa_df["word"], aoa_df["rating_mean"]))

def aoa_score(text, aoa_dict):
    '''Returns the average age of acquisition score for a given text'''
    tokens = tokenize(text)
    aoa_values = [aoa_dict[word] for word in tokens if word in aoa_dict]

    # if there are no words, return a default value
    if len(aoa_values) == 0:
        return np.nan
    
    aoa_score = np.mean(aoa_values)

    return aoa_score

example3 = "because I am cool"
print(aoa_score(example3, aoa_dict))
print('This result should be lower than:')
example4 = "sophisticated technical jargon"
print(aoa_score(example4, aoa_dict))

4.51
This result should be lower than:
11.163333333333334


In [52]:
def zipf_score(text):
    '''Returns a frequency score (higher -> more frequent) based on the Zipf scale'''
    tokens = tokenize(text)
    zipf_values = [zipf_frequency(word, 'en') for word in tokens]

     # if there are no words, return a default value
    if len(zipf_values) == 0:
        return np.nan

    zipf_score = np.mean(zipf_values)

    return zipf_score

print(zipf_score(example3))
print('This result should be higher than:')
print(zipf_score(example4))
print()

example5 = "word"
print(zipf_score(example5))
print('This result should be 5.26')

6.012500000000001
This result should be higher than:
4.013333333333334

5.26
This result should be 5.26


## Computing Syntactic Quality Metrics With Python